In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 856.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.5 MB/s eta 0:00:00


In [ ]:
!pip install cvzone


  Preparing metadata (setup.py) ... done
  Created wheel for cvzone: filename=cvzone-1.6.1-py3-none-any.whl size=26298 sha256=d7321fa4b896a803511beb496a74f2c685315e69f4fae2e371486f8a7f4183ee
  Stored in directory: /root/.cache/pip/wheels/ab/36/ec/47be2d4e59dc4289e684d5b0dde54d1e72e51a614e57690e85
Successfully built cvzone


In [ ]:
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import cvzone

# Load YOLO model
model = YOLO("/content/best.pt")
class_names = model.names

# Load video
cap = cv2.VideoCapture('/content/p.mp4')
count = 0

while True:
    ret, img = cap.read()
    if not ret:
        break

    count += 1
    if count % 3 != 0:
        continue  # Skip frames to reduce processing

    img = cv2.resize(img, (1020, 500))
    h, w, _ = img.shape

    # Run detection
    results = model.predict(img)

    for r in results:
        boxes = r.boxes  # Bounding boxes
        masks = r.masks  # Segmentation masks

        if masks is not None:
            masks = masks.data.cpu().numpy()
            for seg, box in zip(masks, boxes):
                seg = cv2.resize(seg, (w, h))
                contours, _ = cv2.findContours((seg > 0.5).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                for contour in contours:
                    d = int(box.cls)
                    c = class_names[d]
                    x, y, x1, y1 = cv2.boundingRect(contour)

                    # Draw mask contour and bounding box
                    cv2.polylines(img, [contour], True, color=(0, 0, 255), thickness=2)
                    cv2.rectangle(img, (x, y), (x + x1, y + y1), (255, 0, 0), 2)
                    cv2.putText(img, c, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # ✅ Display using Colab-compatible method
    cv2_imshow(img)

cap.release()


In [ ]:


# Load model
model = YOLO("/content/best.pt")
class_names = model.names

# Open input video
cap = cv2.VideoCapture('/content/p.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video writer setup
output_path = "/content/damage_detection_output.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

count = 0

while True:
    ret, img = cap.read()
    if not ret:
        break

    count += 1
    if count % 3 != 0:
        continue  # Frame skipping for speed

    # Run model prediction
    results = model(img, verbose=False)

    for r in results:
        boxes = r.boxes
        masks = r.masks

        if masks is not None:
            mask_data = masks.data.cpu().numpy()
            for seg, box in zip(mask_data, boxes):
                # Resize and binarize the mask
                seg_resized = cv2.resize(seg, (img.shape[1], img.shape[0]))
                mask_binary = (seg_resized > 0.5).astype(np.uint8)

                contours, _ = cv2.findContours(mask_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                for contour in contours:
                    cls_id = int(box.cls)
                    label = class_names[cls_id]

                    x, y, w_box, h_box = cv2.boundingRect(contour)

                    # Draw detection
                    cv2.drawContours(img, [contour], -1, (0, 0, 255), 2)
                    cv2.rectangle(img, (x, y), (x + w_box, y + h_box), (255, 0, 0), 2)
                    cv2.putText(img, f"Damage: {label}", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Write processed frame to output video
    out.write(img)

cap.release()
out.release()
print(f"✅ Damage detection completed! Output saved to: {output_path}")


✅ Damage detection completed! Output saved to: /content/damage_detection_output.mp4
